# Assignment 2A

**Name**: Aditya Nikam                               </br>
**Roll No.**: 210066

In [1]:
# import
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random

In [2]:
'''

Represents a Stochastic Maze problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticMazeEnv(gym.Env):
    '''
    StochasticMazeEnv represents the Gym Environment for the Stochastic Maze environment
    States : [0,1,2,3,4,5,6,7,8,9,10,11]
    Actions : ["Left":0, "Up":1, "Right":2, "Down":3]
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=0,no_states=12,no_actions=4):
        '''
        Constructor for the StochasticMazeEnv class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 12
            no_actions : The no. of possible actions which is 4
            
        '''
        self.initial_state = initial_state
        self.stat1e = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.actions_dict = {"L":0, "U":1, "R":2, "D":3}
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        index = np.random.choice(3,1,p=[0.8,0.1,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

In [3]:
env = StochasticMazeEnv()
env.reset()
num_states = env.nS
num_actions = env.nA


## Test Cases for checking the environment 

## Example: Random Policy

In [4]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    rand_action = np.random.choice(4,1)[0]  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(rand_action)
    
    total_reward += reward

    print("  ", rand_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   1 	   2 	 -0.01 	   False
   0 	   1 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   3 	   8 	 -0.01 	   False
   1 	   4 	 -0.01 	   False
   0 	   8 	 -0.01 	   False
   1 	   4 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   3 	   8 	 -0.01 	   False
   3 	   8 	 -0.01 	   False
   2 	   8 	 -0.01 	   False
   3 	   8 	 -0.01 	   False
   1 	   4 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   2 	   1 	 -0.01 	   False
   0 	   1 	 -0.01 	   False
   3 	   1 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   3 	   4 	 -0.01 	   False
   1 	  

### The random policy takes large number of steps to reach some terminal state which should be much higher than the number of the steps taken by a all 'Right' policy

## Example: Right Policy

In [5]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    right_action = 2  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(right_action)
    
    total_reward += reward

    print("  ", right_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 4
Final Reward: 0.97


### The right policy most of the time reaches the goal state in just 3 steps which is expected

***

## Q1. Find an optimal policy to navigate the given environment using Policy Iteration 

In [4]:
'''This function takes input P(taking action a/ given state s), P(going to state s1/ given current state s), 
value function and expected reward being in state s and taken action a (R)'''
def policy_evaluation(P_a_s, P_s1_s, value, R):    
    d_f = 0.9
    is_not_same = True
    k = 0
    R_pi = np.zeros(12)
    for s in range(0,12):
        R_pi[s] = np.dot(R[s,:],P_a_s[s])
    delta = 0
    v_k = np.zeros(12)
    while delta < 0.000001:
        for s in range(0,12):
            v_k[s] = R_pi[s] + d_f*np.dot(P_s1_s[s,:],value)
            delta = max(delta,np.abs(v_k[s]-value[s]))
            value[s] = v_k[s]
        k = k + 1
    return v_k, R_pi
    

In [5]:
'''THis function takes input P(going to state s1/ given current state s and taken action a), value function,
P(next state s1 / current state s) and expected reward being in state s and taken action a (R)'''
def policy_improvement(P_s1_sa, v_k, P_s1_s, R):
    discount_factor = 0.9
    q_k = np.zeros((12,4))
    for s in range(0,12):
        for a in range(0,4):
            q_k[s,a] = R[s,a] + discount_factor*np.dot(P_s1_sa[s,a,:],v_k)
            
    P_a_s = np.zeros((12,4))    
    index = np.argmax(q_k,axis=1)
    for s in range(0,12):
        P_a_s[s,index[s]] = 1
        
    for s1 in range(0,12):
        for s in range(0,12):
            P_s1_s[s][s1] = np.dot((P_s1_sa[s,:,s1]),(P_a_s[s]))

    return q_k, P_a_s, P_s1_s

In [6]:
'''This function takes input P(next state s1 / given current state s), R and 
P(going to state s1/ given current state s and taken action a)'''
def policy_iteration(R, P_s1_sa):
    value = np.zeros(12)
    v_k1 = value
    P_a_s = [0.25, 0.25, 0.25, 0.25]*np.ones((12,4)) # randomly initiating policy
    P_s1_s = np.zeros((12,12))
    for s1 in range(0,12):
        for s in range(0,12):
            P_s1_s[s][s1] = np.dot((P_s1_sa[s,:,s1]),(P_a_s[s]))
    P_a_s1 = P_a_s
    P_s1_sa1 = P_s1_sa
    P_s1_s1 = P_s1_s
    optimal = False
    while not optimal:
        v_k, R_pi = policy_evaluation(P_a_s1, P_s1_s1, v_k1, R)
        q_k, P_a_s, P_s1_s = policy_improvement(P_s1_sa1, v_k, P_s1_s1, R)
        action = np.argmax(P_a_s, axis=1)
        # print(action)
        if np.array_equal(P_a_s1, P_a_s):
            optimal = True
        v_k1 = v_k
        P_a_s1 = P_a_s
        P_s1_sa1 = P_s1_sa

        
    return P_a_s, v_k, action

In [7]:
# probability of going to state s1 given current state s and action a
P = env.prob_dynamics
P_s1_sa = np.zeros((12,4,12))  # current state, action, next state

for s in range(0,12):
    for a in range(0,4):
        if s == 5:
            break
        tuple = P[s][a]
        sums_dict = {}
        
        # Calculate the sums
        for elem in tuple:
            second_element = elem[1]
            if second_element not in sums_dict:
                sums_dict[second_element] = 0
                P_s1_sa[s][a][second_element] = 0
                
            sums_dict[second_element] += elem[0]
            P_s1_sa[s][a][second_element] += elem[0]


In [8]:
# Probability of getting reward r given current state s and action a
P_r_sa = np.zeros((12,4,3))  # current state, action, reward
for s in range(0,12):
    for a in range(0,4):
        if s == 5:
            break
        tuple = P[s][a]
        sums_dict = {}
        
        # Calculate the sums
        for elem in tuple:
            third_element = elem[2]
            if elem[2] == -0.01:
                r = 0
            if elem[2] == -1:
                r = 1
            if elem[2] == 1:
                r = 2
            if third_element not in sums_dict:
                sums_dict[third_element] = 0
                P_r_sa[s][a][r] = 0
                
            sums_dict[third_element] += elem[0]
            P_r_sa[s][a][r] += elem[0]

# print(P_r_sa)

In [9]:
# expected reward being in state s and taken action a (R) 
r = np.array([-0.01, -1, 1])
R = np.zeros((12,4))
# R[5,:] = -0.01*np.ones(4)
Rr = {}
for s in range(0,12):
    if s == 5:
        continue
    for a in range(0,4):
        R[s][a] = np.dot(r,P_r_sa[s][a])        
# print(R)     

In [10]:
P_A_S, v_k, action =  policy_iteration(R, P_s1_sa)
# print(P_A_S)
# print(v_k)
print(action)

[2 2 2 0 1 0 1 0 2 2 1 0]


## Q2. Find an optimal policy to navigate the given environment using Value Iteration

In [25]:
'''This function takes input P(taking action a/ given state s), P(going to state s1/ given current state s), 
value function and expected reward being in state s and taken action a (R)'''
def value_evaluation(P_a_s, P_s1_s, value, R):
    
    discount_factor = 0.9
    is_not_same = True
    k = 0
    R_pi = np.zeros(12)
    for s in range(0,12):
        R_pi[s] = np.dot(R[s,:],P_a_s[s])

    for s in range(0,12):
        v_k[s] = np.max(R[s,a] + discount_factor*np.dot(P_s1_sa[s,a,:],value))
        value[s] = v_k[s]
    value = v_k
    return v_k, R_pi

In [26]:
'''This function takes input P(going to state s1/ given current state s and taken action a), value function
and expected reward being in state s and taken action a (R)'''
def policy_improvement(P_s1_sa, v_k, P_s1_s, R):
    discount_factor = 0.9
    q_k = np.zeros((12,4))
    for s in range(0,12):
        for a in range(0,4):
            q_k[s,a] = R[s,a] + discount_factor*np.dot(P_s1_sa[s,a,:],v_k)
            
    # print(q_k)
    P_a_s = np.zeros((12,4))
    
    index = np.argmax(q_k,axis=1)
    for s in range(0,12):
        P_a_s[s,index[s]] = 1
    
    
    for s1 in range(0,12):
        for s in range(0,12):
            P_s1_s[s][s1] = np.dot((P_s1_sa[s,:,s1]),(P_a_s[s]))

    return q_k, P_a_s, P_s1_s

In [27]:
'''THis function takes input P(going to state s1/ given current state s), expected reward being
in state s and taken action a (R) and P(going to state s1/ given current state s and taken action a)'''
def value_iteration(R, P_s1_sa):
    value = np.zeros(12)
    v_k1 = value
    P_a_s = [0.25, 0.25, 0.25, 0.25]*np.ones((12,4))
    P_s1_s = np.zeros((12,12))
    for s1 in range(0,12):
        for s in range(0,12):
            P_s1_s[s][s1] = np.dot((P_s1_sa[s,:,s1]),(P_a_s[s]))
    P_a_s1 = P_a_s
    P_s1_sa1 = P_s1_sa
    P_s1_s1 = P_s1_s
    optimal = False
    while not optimal:
        v_k, R_pi = value_evaluation(P_a_s1, P_s1_s1, v_k1, R)
        q_k, P_a_s, P_s1_s = policy_improvement(P_s1_sa1, v_k, P_s1_s, R)
        action = np.argmax(P_a_s, axis=1)
        if np.array_equal(P_a_s1, P_a_s):
            optimal = True
        v_k1 = v_k
        P_a_s1 = P_a_s
        P_s1_sa1 = P_s1_sa
    return P_a_s, v_k, action

In [28]:
# probability of going to state s1 given current state s and action a
P_s1_sa = np.zeros((12,4,12))  # current state, action, next state

for s in range(0,12):
    for a in range(0,4):
        if s == 5:
            break
        tuple = P[s][a]
        sums_dict = {}
        
        # Calculate the sums
        for elem in tuple:
            second_element = elem[1]
            if second_element not in sums_dict:
                sums_dict[second_element] = 0
                P_s1_sa[s][a][second_element] = 0
                
            sums_dict[second_element] += elem[0]
            P_s1_sa[s][a][second_element] += elem[0]


In [29]:
# Probability of reward given current state s and action a
P_r_sa = np.zeros((12,4,3))  # current state, action, reward
for s in range(0,12):
    for a in range(0,4):
        if s == 5:
            break
        tuple = P[s][a]
        sums_dict = {}
        
        # Calculate the sums
        for elem in tuple:
            third_element = elem[2]
            if elem[2] == -0.01:
                r = 0
            if elem[2] == -1:
                r = 1
            if elem[2] == 1:
                r = 2
            if third_element not in sums_dict:
                sums_dict[third_element] = 0
                P_r_sa[s][a][r] = 0
                
            sums_dict[third_element] += elem[0]
            P_r_sa[s][a][r] += elem[0]

# print(P_r_sa)

In [30]:
# expected reward being in state s and taken action a (R) 
r = np.array([-0.01, -1, 1])
R = np.zeros((12,4))
# R[5,:] = -0.01*np.ones(4)
Rr = {}
for s in range(0,12):
    if s == 5:
        continue
    for a in range(0,4):
        R[s][a] = np.dot(r,P_r_sa[s][a])        
# print(R)     

In [32]:
P_A_S, v_k, action =  value_iteration(R, P_s1_sa)
# print(P_A_S)
print(v_k)
print(action)

[-0.03296746 -0.01180231  0.10744032  3.439      -0.03454841  0.
 -0.40238685 -3.439      -0.03473848 -0.03674351 -0.0371464  -0.03721398]
[2 2 2 0 1 0 0 0 0 0 3 3]


## Q3. Compare PI and VI in terms of convergence. Is the policy obtained by both same?

### Answer: 